In [15]:
import os
import glob

import time
import datetime

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [16]:
# 実行結果の日付
log_datetime = '20230608_2335'

In [17]:
RESULT_FILES =  os.path.join('/home', 'jovyan', 'work', 'result', '*.csv')

result_list = []

for file in glob.glob(RESULT_FILES):
    if log_datetime not in file:
        continue
        
    print(file)
    result_list.append(file)
    
    prog_tag = file.replace('.csv', '').split('/')[-1]
    prog_tag = "_".join(prog_tag.split('_')[2:3])
    
print()
print('Prog tag: ' + prog_tag)

/home/jovyan/work/result/20230608_2335_Greedy_01_testset_sys_main.csv

Prog tag: Greedy


In [18]:
CHAMP_DIR = os.path.join('/home', 'jovyan', 'work', 'result', 'champion')

champ_path = os.path.join(CHAMP_DIR, 'champ.csv')
champ_df = pd.read_csv(champ_path)
champ_df_bk = champ_df.copy()

In [19]:
def update_champion(file_name, champ_df):
    result_df = pd.read_csv(file_name)

    result_df = result_df[['seed', 'score', 'result']]

    merge_df = pd.merge(champ_df, result_df, on=['seed'], how='left')

    print('New highest record: {}'.format(sum(merge_df['champion_score'] < merge_df['score'])))
    
    new_record = merge_df['champion_score'] <= merge_df['score']

    merge_df.loc[new_record, 'champion_score'] = merge_df[new_record]['score']
    merge_df.loc[new_record, 'champion_result'] = merge_df[new_record]['result']
    merge_df.loc[new_record, 'champion'] = prog_tag

    merge_df.drop('score', axis=1, inplace=True)
    merge_df.drop('result', axis=1, inplace=True)
    
    return merge_df

In [20]:
for file in result_list:
    champ_df = update_champion(file, champ_df)

New highest record: 198


In [21]:
f_all = champ_df['champion'] == prog_tag
print('New updated record: {} problems'.format(sum(f_all)))

New updated record: 198 problems


## 結果を更新

In [22]:
t_now = datetime.datetime.now() + datetime.timedelta(hours=9)
time_str = t_now.strftime('%Y%m%d_%H%M')
    
if sum(f_all) > 0:
    champ_backup_path = champ_path.replace('.csv', '_' + time_str + '.csv')
    champ_df.to_csv(champ_backup_path, index=False)

    champ_df.to_csv(champ_path, index=False)

## 初期化

In [4]:
if False:
    # テストケースマスタ
    MASTER_PATH = os.path.join('/home', 'jovyan', 'work', '01_testset', '01_testset_sys_master.csv')
    master_df = pd.read_csv(MASTER_PATH)

    master_df['champion'] = ''
    master_df['champion_score'] = 0
    master_df['champion_result'] = ''

    INIT_PATH = os.path.join('/home', 'jovyan', 'work', 'result', 'champion', 'champ_init.csv')
    master_df.to_csv(INIT_PATH, index=False)